# п. 4 - обучение на данных, которые есть в тестовом наборе

In [1]:
import numpy as np
import pandas as pd
import scipy
import math

from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import RidgeClassifier, LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import cross_validate, learning_curve, StratifiedKFold

import matplotlib.pyplot as plt
import random



from collections import Counter

4\. Также вы уже реализовали несколько стратегий для обработки категориальных признаков. Сравните эти стратегии между собой с помощью оценки качества моделей по кросс-валидации, построенных на датасетах с использованием различных стратегий. Как обработка категориальных признаков сказывается на качестве модели? Какой вариант выглядит наиболее оптимальным с точки зрения качества?

In [2]:
def cross_validate_rezult(cv_rez):
    return [
#         type(cv_rez['estimator'][0]).__name__, 
        type(cv_rez['estimator'][0][2]).__name__,
        cv_rez['train_score'].mean(),
        cv_rez['test_score'].mean(),
        len(cv_rez['estimator']),
        cv_rez['fit_time'].mean()            
    ]

In [3]:
df_data = pd.read_csv('orange_small_churn_train_data.csv', index_col='ID')
df_data.rename(columns={"labels": "Label"}, inplace=True)

In [4]:
# Ограничим слова в переменных, которые будем использовать в тренировочном наборе
# теми, что есть в тестовом наборе. Остальные заменим на other
df_enable_words = pd.read_csv('orange_small_churn_test_data.csv', index_col='ID')
for col in df_enable_words.select_dtypes(['object']).columns:
    tmpset = set(df_enable_words[col].unique())
    df_data[col] = df_data[col].apply(lambda x: x if x in tmpset else 'other')

df_enable_words = None

In [5]:
# Удаление столбцов, в которых все NaN. Кроме того, в Label есть NaN.
# отбрасываем и эти строки.
df_data = df_data[np.isfinite(df_data['Label'])].dropna(axis=1, how='all')
# Удаление строк, в которых меньше 2х не NaN переменных (на этом наборе бесполезна)
df_data.dropna(axis=0, how='all', inplace=True, thresh=2)
df_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18298 entries, 0 to 18297
Columns: 213 entries, Var1 to Label
dtypes: float64(174), int64(1), object(38)
memory usage: 29.9+ MB


In [6]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18298 entries, 0 to 18297
Columns: 213 entries, Var1 to Label
dtypes: float64(174), int64(1), object(38)
memory usage: 29.9+ MB


In [7]:
# df_data = pd.read_csv('not_hold_out_dataset.csv', index_col=0)
cat_var = df_data.select_dtypes(['object']).columns
cat_num = df_data.select_dtypes(exclude=['object']).columns[:-1]

In [8]:
def CalcCorr(Y, X):
    iY0 = np.nonzero(Y==-1)
    iY1 = np.nonzero(Y==1)
    iX0 = np.nonzero(X==0)
    iX1 = np.nonzero(X==1)
#     print(iY0, iY1, iX0, iX1)
    a = 1.*len(np.intersect1d(iY0, iX0))
    b = 1.*len(np.intersect1d(iY0, iX1))
    c = 1.*len(np.intersect1d(iY1, iX0))
    d = 1.*len(np.intersect1d(iY1, iX1))
    t = (a+b)*(c+d)*(a+c)*(b+d)
    if t == 0.:
        ret = 0
        print()
#         print(iY0, iY1, iX0, iX1)
        print('a={}, b={}, c={}, d={}'.format(a, b, c, d))
    else: 
        ret = (a*d-b*c)/(math.sqrt((a+b)*(c+d)*(a+c)*(b+d) ))
    return ret

# Y = np.array([-1, -1, -1, 1, 1, 1, 1], dtype='int8')
# X = np.array([ 0,  0,  0, 0, 1, 0, 0], dtype='int8')
# CalcCorr(Y, X)   
# np.nonzero(Y==-1)

In [9]:
%%time
# Методика отбора признаков - по наибольшей корреляции

# Вычисление корреляции каждой переменной в каждом категориальном параметре
# Цель - в дальнейшем оставить переменные и значения с наибольшей корреляцией
# или коррелирующиеся.
# Так как расчет выполняется несколько минут (только что - 7 минут, с введением фильтра 
# по количеству переменных равных 50 - 12 сек.),
# то данные сохранены в файл
acorr = []
aLabels = np.array(df_data['Label'].values)
# cat_var=['Var200']
for c in cat_var:
    
    dVals = df_data[c].value_counts().to_dict()
    df_dummies=pd.get_dummies(df_data[c])
    print(c, len(dVals), end=' ')
    d = np.array(df_dummies.values)
    for i, c_val in enumerate(df_dummies.columns):
#         if dVals[c_val] > 50:
        rez = CalcCorr(aLabels, d[:,i])
#             rez = CalcCorr(aLabels, np.array(df_dummies[c_val].values))
        acorr.append([c, c_val, dVals[c_val], rez])

            # df_data['Var193']
df_corr_categ = pd.DataFrame(acorr, columns=['Var', 'Val', 'Count', 'correlation'], index=None)
df_corr_categ.to_pickle('df_corr_categ_kaggle_01.bin')

Var191 1 Var192 265 Var193 40 Var194 3 Var195 17 Var196 4 Var197 177 Var198 1460 Var199 906 Var200 1401 Var201 2 Var202 3224 Var203 3 Var204 100 Var205 3 Var206 21 Var207 12 Var208 2 Var210 6 Var211 2 Var212 63 Var213 1 Var214 1401 Var215 1 Var216 640 Var217 2891 Var218 2 Var219 15 Var220 1460 Var221 7 Var222 1460 Var223 4 Var224 1 Var225 3 Var226 23 Var227 7 Var228 26 Var229 4 Wall time: 50.9 s


In [10]:
df_corr_categ = pd.read_pickle('df_corr_categ_kaggle_01.bin')
df_corr_categ['correlation_abs'] = df_corr_categ['correlation'].abs()

In [14]:
df[df['Val'] == 'other']

,Var,Val,Count,correlation,correlation_abs


In [11]:
df = df_corr_categ[df_corr_categ['Val'] != 'other'].sort_values(by=['correlation_abs'], ascending =False)
df = df[(df['Count'] > 3) & (df['correlation_abs'] > 0.012)]

# Строка ниже подобрана для 'not_hold_out_dataset.csv'
# df = df[(df['Count'] > 0) & (df['correlation_abs'] > 0.0187)]
# При df[df['correlation_abs'] > 0.0188] плучено значение  roc-auc = 0.7491
# с классификатором GradientBoostingClassifier
# df = df[df['correlation_abs'] > 0.018]
df.shape

(717, 5)

In [12]:
%%time
df_cat_var = df_data[df['Var'].unique()].copy()
for col in df_cat_var.columns:
    val_set = set(df[ df['Var'] == col]['Val'].values)
    df_cat_var[col] = df_cat_var[col].apply(lambda x: x if x in val_set else 'other')

Wall time: 233 ms


In [13]:
df_cat_var['Var212'].value_counts()
# df_data['Var202'].value_counts()

NhsEn4L          10687
other             2660
XfqtO3UdzaXh_     2359
CrNX              1127
Ie_5MZs            527
4kVnq_T26xq1p      377
_5OXC8MSLt         262
h0lfDKh52u4GP      260
TW8UhnX             39
Name: Var212, dtype: int64

In [14]:
df_cat_var_dummies = pd.get_dummies(df_cat_var)
# df_cat_var_dummies.info()

In [15]:
df_test = pd.concat([df_data[cat_num], df_cat_var_dummies ], axis=1)
X = df_test.values
# Массив с метками.
y = df_data['Label'].values
# df_test = None

In [16]:
X.shape

(18298, 922)

In [17]:
# len(cat_num)  
len(df_cat_var_dummies.columns)

748

In [18]:
# df_cat_var_dummies = None
# df_cat_var = None
# df_data = None

In [19]:
%%time
train_score, test_score = [], []

cv = StratifiedKFold(n_splits=10, shuffle=True)   
si = SimpleImputer(strategy='median')
# 'median', 'most_frequent' mean
scaler = preprocessing.MinMaxScaler()
# scaler = preprocessing.StandardScaler()
rez = []
for est in [
#     SGDClassifier(), 
#     LinearSVC(),
    LogisticRegression( solver='liblinear' ),
#     RidgeClassifier(),
#     RandomForestClassifier(), 
#     SVC(kernel='linear'),
#     GradientBoostingClassifier()
]:
    pipe = Pipeline([
        ('inputer', si), 
        ('scaler', scaler), 
        ('est', est)
    ])
    print(type(est).__name__, end=' ')
    cv_rez = cross_validate(pipe, X, y, cv=cv, 
                          return_train_score=True, 
                          return_estimator=True,
    
                            verbose=0, n_jobs=4,
                          scoring=('roc_auc'))
    print(cv_rez['test_score'].mean().round(4))
    rez.append(cross_validate_rezult(cv_rez))
#     train_score.append(cv_rez['train_score'])
#     test_score.append(cv_rez['test_score'])
#     print(cv_rez['test_score'].mean().round(4), cv_rez['train_score'].mean().round(4), type(est).__name__)

LogisticRegression 0.7554
Wall time: 40.9 s


In [25]:
cv_rez

{'fit_time': array([10.27931046,  9.86524796, 10.31057787,  9.258816  ,  9.68277955,
        12.92658329, 11.91898823, 11.19873929,  7.35449195,  6.36621094]),
 'score_time': array([0.09474993, 0.08812022, 0.13833666, 0.10612583, 0.15217566,
        0.09570289, 0.09472632, 0.11328125, 0.06250024, 0.046875  ]),
 'estimator': (Pipeline(memory=None,
           steps=[('inputer',
                   SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                                 missing_values=nan, strategy='median',
                                 verbose=0)),
                  ('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))),
                  ('est',
                   LogisticRegression(C=1.0, class_weight=None, dual=False,
                                      fit_intercept=True, intercept_scaling=1,
                                      l1_ratio=None, max_iter=100,
                                      multi_class='warn', n_jobs=None,
                         

In [27]:
rez_df = pd.DataFrame(rez, columns=['Алгоритм классификации', 
                                   'ROC-AUC train', 'ROC-AUC test', 
                                   'Количество фолдов', 'Время обучения фолда'])
rez_df = rez_df.round(decimals=4).sort_values(by=['ROC-AUC test'])
rez_df

,Алгоритм классификации,ROC-AUC train,ROC-AUC test,Количество фолдов,Время обучения фолда
0,LogisticRegression,0.7678,0.7393,10,8.2700
1,GradientBoostingClassifier,0.8132,0.7521,10,201.0516


### Предсказание

In [20]:
df_kaggle = pd.read_csv('orange_small_churn_test_data.csv', index_col='ID')
df_kaggle.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230
ID,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,1225.0,7.0,NaN,NaN,NaN,...,zCkv,APgdzOv,jySVZNlOJy,NaN,ELof,xb3V,6fzt,Zy3gnGM,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,896.0,14.0,NaN,NaN,NaN,...,oslk,IIvC99a,LM8l689qOp,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,791.0,7.0,NaN,NaN,NaN,...,oslk,6YSocsg,LM8l689qOp,NaN,kG3k,rgKb,RAYp,F2FyR07IdsN7I,mj86,NaN
3,NaN,NaN,NaN,NaN,NaN,2296.0,7.0,NaN,NaN,NaN,...,oslk,5nQ7A2G,jySVZNlOJy,NaN,kG3k,rgKb,RAYp,F2FyR07IdsN7I,am7c,NaN
4,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,NaN,...,oslk,MI8s5nE,LM8l689qOp,NaN,NaN,7P5s,RAYp,F2FyR07IdsN7I,NaN,NaN


In [21]:
# Для получения необходимой размерности, создадим пустую матрицу, с количеством колонок
# равным количеству колонок в тренировочной выборке
df_test_kagle = pd.DataFrame(np.full((df_kaggle.shape[0], len(df_cat_var_dummies.columns)), np.nan), 
                  columns=df_cat_var_dummies.columns)
df_test_kagle.head()

,Var205_09_Q,Var205_VpdQ,Var205_other,Var205_sJzTlal,Var212_4kVnq_T26xq1p,Var212_CrNX,Var212_Ie_5MZs,Var212_NhsEn4L,Var212_TW8UhnX,Var212_XfqtO3UdzaXh_,...,Var196_1K8T,Var196_other,Var196_z3mO,Var213_KdSa,Var213_other,Var203_9_Y1,Var203_F3hy,Var203_other,Var191_other,Var191_r__I
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
%%time
# Отберем столбцы, которые были в тренинговом наборе
df_cat_var = df_kaggle[df['Var'].unique()].copy()
# Переделать, с учетом того, что должны быть и столбцы, которых в тестовом наборе и не получим
# учесть и порядок
for col in df_cat_var.columns:
    val_set = set(df[ df['Var'] == col]['Val'].values)
    df_cat_var[col] = df_cat_var[col].apply(lambda x: x if x in val_set else 'other')

df_cat_var_dummies_kaggle = pd.get_dummies(df_cat_var)
# В этот момент набор df_cat_var_dummies_kaggle содержит не больше столбцов, чем тренинговый набор,
# но может содержать меньше (это наиболее вероятно, так как каких-то значений, которые
# были в тренинговом наборе, в тестовом может не быть)
for col in df_cat_var_dummies_kaggle.columns:
    df_test_kagle[col] = df_cat_var_dummies_kaggle[col]
    


df_test_kaggle_conc = pd.concat([df_kaggle[cat_num], df_test_kagle ], axis=1)
X_kaggle = df_test_kaggle_conc.values

Wall time: 17.7 s


In [23]:
X_kaggle.shape

(10000, 922)

In [24]:
%%time
si = SimpleImputer(strategy='median')
scaler = preprocessing.MinMaxScaler()
est = LogisticRegression( solver='liblinear')
# est = GradientBoostingClassifier()
pipe = Pipeline([
    ('inputer', si), 
    ('scaler', scaler), 
    ('est', est)
])

pipe.fit(X, y)


Wall time: 4.54 s


Pipeline(memory=None,
         steps=[('inputer',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                               missing_values=nan, strategy='median',
                               verbose=0)),
                ('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))),
                ('est',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='warn', n_jobs=None,
                                    penalty='l2', random_state=None,
                                    solver='liblinear', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=False)

In [25]:
y_kaggle = pipe.predict_proba(X_kaggle)

df_kaggle['result'] = y_kaggle[:,1]
df_kaggle[['result']].round(2).to_csv('dminik6_part2_w3_09.csv')

dminik6_part2_w3_05.csv на Kaggle дал 0.70036.
е
Данные для обучения:  менялись только строковые переменные. После dummies отбрасывались 
те, у которых коэффициент корреляции с целевой переменной был меньше 0.02 и чье количество было больше 3, такие переменные заменены на 'other'. Отобрано 249 переменных.


Обучение на pipe: sSimpleImputer(strategy='median'), preprocessing.MinMaxScaler(), GradientBoostingClassifier(). На тренинговом наборе получено 0.7518 (10 фолдов)

## threshold = 30

0.6801 0.738 LogisticRegression  
0.6736 0.7365 RidgeClassifier

Без preprocessing.MinMaxScaler()  
0.5962 0.6077 LogisticRegression  
0.6726 0.7372 RidgeClassifier  
Wall time: 7min 22s


scaler = preprocessing.StandardScaler()  
0.6685 0.7465 LogisticRegression  
0.6697 0.7376 RidgeClassifier  
Wall time: 10min 41s


si = SimpleImputer(strategy='median')  
0.6791 0.7391 RidgeClassifier  
Wall time: 5min 18s


si = SimpleImputer(strategy='median')  
0.6833 0.7411 LogisticRegression  
Wall time: 5min


## threshold = 50
si = SimpleImputer(strategy='median')  
0.684 0.7549 LogisticRegression  
0.6798 0.7537 RidgeClassifier  
Wall time: 17min 5s  



5\. Все ли признаки оказались полезными для построения моделей? Проведите процедуру отбора признаков, попробуйте разные варианты отбора (обратите внимание на модуль `sklearn.feature_selection`). Например, можно выбрасывать случайные признаки или строить отбор на основе l1-регуляризации - отфильтровать из обучения признаки, которые получат нулевой вес при построении регрессии с l1-регуляризацией (`sklearn.linear_model.Lasso`). И всегда можно придумать что-то своё=) Попробуйте как минимум 2 различные стратегии, сравните результаты. Помог ли отбор признаков улучшить качество модели? Поясните свой ответ.

6\. Подберите оптимальные параметры модели. Обратите внимание, что в зависимости от того, как вы обработали исходные данные, сделали ли балансировку классов, сколько объектов оставили в обучающей выборке и др. оптимальные значения параметров могут меняться. Возьмите наилучшее из ваших решений на текущий момент и проведите процедуру подбора параметров модели (обратите внимание на `sklearn.model_selection.GridSearchCV`) Как подбор параметров повлиял на качество модели?

7\. Предложите методику оценки того, какие признаки внесли наибольший вклад в модель (например, это могут быть веса в случае регрессии, а также большое количество моделей реализуют метод `feature_importances_` - оценка важности признаков). На основе предложенной методики проанализируйте, какие признаки внесли больший вклад в модель, а какие меньший?

8\. Напоследок давайте посмотрим на объекты. На каких объектах достигается наибольшая ошибка классификации? Есть ли межу этими объектами что-то общее? Видны ли какие-либо закономерности? Предположите, почему наибольшая ошибка достигается именно на этих объектах. В данном случае "наибольшую" ошибку можно понимать как отнесение объекта с чужому классу с большой долей уверенности (с высокой вероятностью).

9\. По итогам проведенных экспериментов постройте финальную решение - модель с наилучшим качеством. Укажите, какие преобразования данных, параметры и пр. вы выбрали для построения финальной модели.

10\. Подумайте, можно ли еще улучшить модель? Что для этого можно сделать? 